In [11]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import datetime

import numpy as np
from time import time

from sklearn import metrics
from sklearn import preprocessing

from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')



def parse_location(loc):
    loc = loc.strip("()").split(',')
    lat = loc[0].strip()
    long = loc[1].strip()
    return float(lat), float(long)

def classification_report(y_true, y_pred, verbose=False):
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    accuracy_score = metrics.accuracy_score(y_true, y_pred)
    recall_score = metrics.recall_score(y_true, y_pred)
    precision_score = metrics.precision_score(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    
    if verbose:
        print('confusion_matrix')
        print(confusion_matrix)
        print('accuracy_score', accuracy_score)
        print('recall_score', recall_score)
        print('precision_score', precision_score)
        print('f1_score', f1_score)
        print('auc', auc)
        print('null_accuracy', (len(y_true)-sum(y_true))/len(y_true))
    
    return confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, auc
    

def get_scaler(column):
    X = np.array(column).reshape(column.shape[0], 1)
    return preprocessing.MinMaxScaler().fit(column)

def scale(scaler, column):
    X = np.array(column).reshape(column.shape[0], 1)
    return scaler.transform(X)

In [12]:
def scale_data(X,y):

    global scaler_X
    global scaler_y
    
    scaler_X = preprocessing.MinMaxScaler()
    #scaled_X = pd.DataFrame(scaler_X.fit_transform(X), columns=['week_number','day_number',hour_column,'previous_4_weeks_count', 'previous_3_weeks_count','previous_2_weeks_count','previous_1_week_count','previous_day_count'])
    scaled_X = pd.DataFrame(X, columns=X.columns)
    
    scaler_y = preprocessing.MinMaxScaler()
    scaled_y = pd.DataFrame(scaler_y.fit_transform(y.values.reshape(-1,1)))
    
    return scaled_X, scaled_y


def unscale_data(scaled_y):
    global scaler_y
    return scaler_y.inverse_transform(scaled_y)

In [13]:
%time df = pd.read_csv('features_temporal_full_year_with_weather_2500_final.tsv', sep='\t', parse_dates=['timestamp'])
df.index = df.timestamp.apply(lambda x: x.month)
df.head()

CPU times: user 2.02 s, sys: 352 ms, total: 2.37 s
Wall time: 2.38 s


,cell_range,timestamp,crime_freq,yelp_freq,police_freq,lat,long,police_factor,yelp_factor,prev_7_days_crime_freq,prev_day_crime_freq,PRCP,SNOW,TMAX,TMIN
timestamp,,,,,,,,,,,,,,,
1,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-01-01,0,0,0,41.554389,-88.358527,7.041843e-26,0.019742,NaN,NaN,0.8,0,7.2,-3.3
1,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-01-02,0,0,0,41.554389,-88.358527,7.041843e-26,0.019742,NaN,0,9.4,0,5.6,4.4
1,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-01-03,0,0,0,41.554389,-88.358527,7.041843e-26,0.019742,NaN,0,0.0,0,5.6,3.9
1,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-01-04,0,0,0,41.554389,-88.358527,7.041843e-26,0.019742,NaN,0,0.5,0,6.1,2.2
1,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-01-05,0,0,0,41.554389,-88.358527,7.041843e-26,0.019742,NaN,0,0.0,0,2.8,-0.6


In [14]:
df.dropna(inplace=True)

In [24]:
test = None
def train_and_test(X_train, X_test, y_train, y_test, features=None, verbose=False, random_state = 4, classifiers=None):
    def scale_columns(train, test, column_names):
        for column_name in column_names:
            scaler = get_scaler(train[column_name])
            train[column_name] = scale(scaler, train[column_name])
            test[column_name] = scale(scaler, test[column_name])
    
    scale_columns(X_train, X_test, features)
    yScaler = get_scaler(y_train)
    y_train = yScaler.transform(y_train)
    y_test = yScaler.transform(y_test)
    
    global test
    test = X_test.copy() 
    test['label'] = y_test
    
    X_train = X_train[features]
    X_test = X_test[features]
    
    # class_weight = {
    #     1: 10,
    #     0: 1
    # }
    training_size = len(X_train.index.unique())
    
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    
    y_test = yScaler.inverse_transform(y_test)
    y_test_pred = yScaler.inverse_transform(y_test_pred)
    
    test['pred_label'] = y_test_pred
    test[[
                'timestamp',
                'lat',
                'long',
                'police_factor', 
                'yelp_factor', 
                'prev_day_crime_freq',
                'prev_7_days_crime_freq',
                'pred_label',
                'label'
            ]].to_csv('Regression/%s_%s.csv' % ('Regression', training_size), index=False)
        
        
        
    print(metrics.mean_squared_error(y_test, y_test_pred))
    print(metrics.mean_absolute_error(y_test, y_test_pred)) 

In [25]:
def test_models(features, splits=2, verbose=False, classifiers=None):
    if verbose:
        print()
        print('#'*100)
        print('#'*10 + str(features) + '#'*10)
        print('#'*100)
        print()

    X = df
    y = df.crime_freq
    
    tscv = TimeSeriesSplit(n_splits=splits)
    
    frames = []
    for train_index, test_index in tscv.split(X.index.unique()):
#         print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[X.index.isin(train_index+1)], X[X.index.isin(test_index+1)]
        y_train, y_test = y[y.index.isin(train_index+1)], y[y.index.isin(test_index+1)]
#         print(X_train.timestamp.values[0], X_train.timestamp.values[-1], X_test.timestamp.values[0], X_test.timestamp.values[-1])
    
        train_start_date, train_end_date = X_train.timestamp.values[0], X_train.timestamp.values[-1]
        test_start_date, test_end_date = X_test.timestamp.values[0], X_test.timestamp.values[-1]
        training_size = len(X_train.index.unique())
        
        if verbose:
            print()
            print('>'*100)
            print("Training: ", str(train_start_date), "to" , str(train_end_date))
            print("Test: ", str(test_start_date), "to" , str(test_end_date))
            print('>'*100)
            print()
        
        train_and_test(X_train, X_test, y_train, y_test, features=features, verbose=verbose) 

In [26]:
features = [
#     'PRCP', 
#     'SNOW', 
#     'TMAX', 
#     'TMIN',
#     'crime_freq', 
#      'yelp_freq', 
#      'police_freq', 
     'police_factor', 
#     'crime_factor', 
    'yelp_factor', 
    'prev_day_crime_freq',
    'prev_7_days_crime_freq'
]

report_df = test_models(features, splits=11, verbose=True)


####################################################################################################
##########['police_factor', 'yelp_factor', 'prev_day_crime_freq', 'prev_7_days_crime_freq']##########
####################################################################################################


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Training:  2006-01-08T00:00:00.000000000 to 2006-01-31T00:00:00.000000000
Test:  2006-02-01T00:00:00.000000000 to 2006-02-28T00:00:00.000000000
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

0.642940581395
0.188472636335

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Training:  2006-01-08T00:00:00.000000000 to 2006-02-28T00:00:00.000000000
Test:  2006-03-01T00:00:00.000000000 to 2006-03-31T00:00:00.000000000
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [27]:
test

,cell_range,timestamp,crime_freq,yelp_freq,police_freq,lat,long,police_factor,yelp_factor,prev_7_days_crime_freq,prev_day_crime_freq,PRCP,SNOW,TMAX,TMIN,label,pred_label
timestamp,,,,,,,,,,,,,,,,,
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-01,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,31.2,147,0.6,-7.8,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-02,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,-1.1,-11.1,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-03,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,-9.4,-13.3,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-04,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,-3.3,-14.4,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-05,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,1.1,-13.9,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-06,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,1.7,-8.3,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-07,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,-8.3,-12.8,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-08,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,-6.1,-15.6,0,-0.007356
12,"((41.5487, -88.3713), (41.560078, -88.345754))",2006-12-09,0,0,0,41.554389,-88.358527,0.000000e+00,1.548110e-07,0,0,0.0,0,1.7,-6.7,0,-0.007356
